# RAG-агент: поиск терапевтических мишеней болезни Альцгеймера

**Тестовое задание — Стажёры 2026**

---

## Описание проекта

Болезнь Альцгеймера (AD) — наиболее распространённая форма деменции, затрагивающая более 55 миллионов человек по всему миру. Несмотря на десятилетия исследований, одобренных заболевание-модифицирующих терапий крайне мало, а фундаментальное понимание патогенеза продолжает развиваться. Одним из ключевых этапов разработки лекарств является **идентификация и валидация терапевтических мишеней** — белков, рецепторов, ферментов и сигнальных путей, воздействие на которые может замедлить или остановить нейродегенерацию.

Данный проект реализует **RAG-агента** (Retrieval-Augmented Generation), который:

1. **Автоматически собирает** научные статьи из PubMed по релевантным запросам
2. **Строит базу знаний** с семантическим и ключевым поиском (hybrid search)
3. **Отвечает на вопросы** исследователей с указанием конкретных источников `[PMID:XXXXX]`
4. **Оценивает качество** ответов по нескольким метрикам, включая retrieval-метрики на gold standard

### Архитектура

```
PubMed API → XML Parsing → Text Cleaning → Chunking
                                              |
                                    Sentence-Transformers → ChromaDB (dense)
                                              +
                                         BM25 (sparse)
                                              |
                            Hybrid Search (RRF) → Cross-Encoder Re-ranking
                                              |
                                    Local LLM (HuggingFace) → Answer + Citations
```

---

## Содержание

| # | Часть | Описание |
|---|-------|----------|
| 1 | [Подготовка данных](#part1) | Сбор статей с PubMed и PMC, очистка, retry-логика |
| 2 | [EDA](#part2) | Анализ корпуса: распределения, TF-IDF, co-occurrence мишеней |
| 3 | [Векторная БД](#part3) | Parent-child чанкинг, эмбеддинги, ChromaDB + BM25 |
| 4 | [RAG Pipeline](#part4) | Hybrid retrieval → Cross-encoder → LLM → Метрики + Gold standard |
| 5 | [Интерфейс](#part5) | Интерактивные виджеты для исследователей |
| 6 | [Теор. вопросы](#part6) | Модальности, архитектура, обоснование выбора |


In [ ]:
!pip install -q \
    requests \
    matplotlib \
    seaborn \
    wordcloud \
    scikit-learn \
    pandas \
    numpy \
    chromadb \
    sentence-transformers \
    langchain-text-splitters \
    transformers \
    torch \
    accelerate \
    ipywidgets \
    rank-bm25 \
    tiktoken \
    ipython

## Импорты

In [ ]:
import json, time, re, os, gc, hashlib, unicodedata, warnings, shutil
import xml.etree.ElementTree as ET
from pathlib import Path
from dataclasses import dataclass, field, asdict
from collections import Counter, OrderedDict

import requests as req
import math
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer
from wordcloud import WordCloud

import torch
import chromadb
from chromadb.config import Settings
from sentence_transformers import SentenceTransformer, CrossEncoder
from langchain_text_splitters import RecursiveCharacterTextSplitter
from rank_bm25 import BM25Okapi
import tiktoken

import ipywidgets as widgets
from IPython.display import display, Markdown, HTML, clear_output

warnings.filterwarnings('ignore')
%matplotlib inline
sns.set_theme(style='whitegrid', font_scale=1.1)
plt.rcParams['figure.dpi'] = 150

<a id="part1"></a>
# Часть 1. Сбор и подготовка данных

## Методология сбора

Для построения базы знаний используется **NCBI E-utilities API** — программный интерфейс к PubMed (>36 млн записей). Реализация без внешних библиотек для PubMed — чистый `requests` + `xml.etree.ElementTree`.

### Стратегия поиска

Используем **15 поисковых запросов** с разным фокусом. `MAX_PER_QUERY = 50` даёт до 750(636) кандидатов до дедупликации — достаточный корпус для надёжного RAG.

### Pipeline обработки

```
ESearch (JSON) → список PMID
       |
EFetch (XML) → метаданные + абстракты
       |
PMC Full-text (XML) → Introduction + Conclusion
       |
Очистка: inline-ссылки [1,2], (Smith et al.), URL, спецсимволы
       |
Фильтрация: минимум 100 символов + >= 2 ключевых слова
       |
data/clean_articles.json
```


### 1.1 Конфигурация

In [ ]:
EMAIL = 'email@example.com'
TOOL_NAME = 'alzheimer_rag_agent'

BASE_URL = 'https://eutils.ncbi.nlm.nih.gov/entrez/eutils'
ESEARCH_URL = f'{BASE_URL}/esearch.fcgi'
EFETCH_URL  = f'{BASE_URL}/efetch.fcgi'

QUERIES = [
    "Alzheimer's disease targets",
    "Alzheimer therapeutic targets",
    "Alzheimer drug targets",
    "Alzheimer's disease drug discovery targets",
    "novel targets Alzheimer treatment",
    "amyloid beta therapeutic target Alzheimer",
    "tau phosphorylation drug target Alzheimer",
    "neuroinflammation targets Alzheimer's disease",
    "TREM2 Alzheimer therapeutic",
    "BACE1 inhibitor Alzheimer clinical",
    "autophagy target Alzheimer neurodegeneration",
    "synaptic dysfunction Alzheimer drug target",
    "gut-brain axis Alzheimer therapeutic",
    "epigenetic targets Alzheimer's disease",
    "mitochondrial dysfunction Alzheimer drug",
]

MAX_PER_QUERY = 50
OUTPUT_DIR = Path('data')
RAW_JSON   = OUTPUT_DIR / 'raw_articles.json'
CLEAN_JSON = OUTPUT_DIR / 'clean_articles.json'
STATS_JSON = OUTPUT_DIR / 'collection_stats.json'

print(f'Запросов: {len(QUERIES)}')
print(f'Max на запрос: {MAX_PER_QUERY}')
print(f'Потенциальный охват: до {len(QUERIES) * MAX_PER_QUERY} статей (до дедупликации)')


### 1.2 Датакласс `Article`

In [ ]:
@dataclass
class Article:
    pmid: str
    title: str = ''
    authors: list[str] = field(default_factory=list)
    journal: str = ''
    year: str = ''
    doi: str = ''
    abstract: str = ''
    introduction: str = ''
    conclusion: str = ''
    pmc_id: str = ''
    query_source: str = ''
    full_text_available: bool = False


### 1.3 HTTP-хелпер с retry

### `api_get`

HTTP GET-запрос к NCBI API с автоматическим retry при ошибках и rate-limit.

**Вход:** `url` (str), `params` (dict), `timeout` (int), `max_retries` (int)  
**Выход:** `requests.Response` — ответ сервера

In [ ]:
def api_get(url: str, params: dict, timeout: int = 30, max_retries: int = 3):
    params = {**params, 'email': EMAIL, 'tool': TOOL_NAME}
    last_exc = None
    for attempt in range(max_retries):
        try:
            resp = req.get(url, params=params, timeout=timeout)
            if resp.status_code == 429:
                wait = 2 ** attempt
                print(f'  [rate limit] ожидание {wait}s (попытка {attempt+1})')
                time.sleep(wait)
                continue
            resp.raise_for_status()
            return resp
        except req.exceptions.RequestException as exc:
            last_exc = exc
            wait = 2 ** attempt
            print(f'  [retry {attempt+1}/{max_retries}] {exc} -> ожидание {wait}s')
            time.sleep(wait)
    raise RuntimeError(f'API недоступен после {max_retries} попыток: {last_exc}')

### 1.4 Поиск PMID

### `search_pubmed`

Ищет PMID статей в PubMed по текстовому запросу через ESearch API.

**Вход:** `query` (str) — поисковый запрос, `max_results` (int)  
**Выход:** `list[str]` — список найденных PMID

In [ ]:
def search_pubmed(query: str, max_results: int = 100) -> list[str]:
    resp = api_get(ESEARCH_URL, {
        'db': 'pubmed', 'term': query, 'retmax': max_results,
        'sort': 'relevance', 'datetype': 'pdat',
        'mindate': '2010', 'maxdate': '2026', 'retmode': 'json',
    })
    pmids = resp.json().get('esearchresult', {}).get('idlist', [])
    print(f'  "{query}" -> {len(pmids)} PMID')
    return pmids

### 1.5 Сбор уникальных PMID

### `collect_all_pmids`

Собирает уникальные PMID по всем поисковым запросам из QUERIES, дедуплицируя результаты.

**Вход:** нет (использует глобальный QUERIES)  
**Выход:** `dict[str, str]` — словарь {pmid: query_source}

In [ ]:
def collect_all_pmids() -> dict[str, str]:
    pmid_to_query = {}
    for q in QUERIES:
        for pmid in search_pubmed(q, MAX_PER_QUERY):
            if pmid not in pmid_to_query:
                pmid_to_query[pmid] = q
        time.sleep(0.4)
    print(f'\nУникальных PMID: {len(pmid_to_query)}')
    return pmid_to_query

### 1.6 Загрузка PubMed XML

### `fetch_pubmed_xml`

Загружает XML-метаданные статей пакетами через EFetch API.

**Вход:** `pmids` (list[str]) — список PMID, `batch_size` (int)  
**Выход:** `list[Element]` — список XML-элементов PubmedArticle

In [ ]:
def fetch_pubmed_xml(pmids: list[str], batch_size: int = 100) -> list:
    all_articles = []
    for i in range(0, len(pmids), batch_size):
        batch = pmids[i:i + batch_size]
        print(f'  Загрузка {i+1}-{i+len(batch)} из {len(pmids)}')
        resp = api_get(EFETCH_URL, {
            'db': 'pubmed', 'id': ','.join(batch),
            'rettype': 'xml', 'retmode': 'xml',
        })
        root = ET.fromstring(resp.content)
        all_articles.extend(root.findall('.//PubmedArticle'))
        time.sleep(0.4)
    return all_articles

### 1.7 Парсинг PubMed XML

### `_get_text`, `_get_all_text`, `parse_pubmed_article`, `parse_all_articles`

- `_get_text` — извлекает текст из XML-элемента по XPath. **Вход:** Element, path. **Выход:** str
- `_get_all_text` — собирает весь текст из элемента рекурсивно. **Вход:** Element. **Выход:** str
- `parse_pubmed_article` — парсит один PubmedArticle XML-элемент в словарь метаданных. **Вход:** XML Element. **Выход:** dict | None
- `parse_all_articles` — парсит список XML-элементов в список Article. **Вход:** list[Element], dict pmid→query. **Выход:** list[Article]

In [ ]:
def _get_text(el, path, default=''):
    e = el.find(path)
    return e.text.strip() if e is not None and e.text else default

def _get_all_text(el):
    if el is None: return ''
    return ' '.join(c.strip() for c in el.itertext() if c.strip())

def parse_pubmed_article(elem) -> dict | None:
    citation = elem.find('MedlineCitation')
    article = citation.find('Article') if citation is not None else None
    if citation is None or article is None: return None
    pmid = _get_text(citation, 'PMID')
    title = _get_all_text(article.find('ArticleTitle'))
    authors = []
    al = article.find('AuthorList')
    if al is not None:
        for a in al.findall('Author'):
            last = _get_text(a, 'LastName')
            if last: authors.append(f"{last} {_get_text(a, 'Initials')}")
    journal = ''
    je = article.find('Journal')
    if je is not None:
        journal = _get_text(je, 'Title') or _get_text(je, 'ISOAbbreviation')
    year = ''
    pd_ = article.find('.//Journal/JournalIssue/PubDate')
    if pd_ is not None:
        year = _get_text(pd_, 'Year')
        if not year:
            md_ = _get_text(pd_, 'MedlineDate')
            m = re.match(r'(\d{4})', md_) if md_ else None
            if m: year = m.group(1)
    if not year:
        ad = article.find('ArticleDate')
        if ad is not None: year = _get_text(ad, 'Year')
    abstract_parts = []
    ae = article.find('Abstract')
    if ae is not None:
        for at in ae.findall('AbstractText'):
            label = at.get('Label', '')
            text = _get_all_text(at)
            if text: abstract_parts.append(f'{label}: {text}' if label else text)
    abstract = ' '.join(abstract_parts)
    doi = ''
    for eloc in article.findall('ELocationID'):
        if eloc.get('EIdType') == 'doi' and eloc.text: doi = eloc.text.strip(); break
    if not doi:
        pdata = elem.find('PubmedData')
        if pdata:
            for aid in pdata.findall('.//ArticleId'):
                if aid.get('IdType') == 'doi' and aid.text: doi = aid.text.strip(); break
    pmc_id = ''
    pdata = elem.find('PubmedData')
    if pdata:
        for aid in pdata.findall('.//ArticleId'):
            if aid.get('IdType') == 'pmc' and aid.text: pmc_id = aid.text.strip(); break
    return {'pmid': pmid, 'title': title, 'authors': authors, 'journal': journal,
            'year': year, 'doi': doi, 'abstract': abstract, 'pmc_id': pmc_id}

def parse_all_articles(xml_articles, pmid_to_query) -> list[Article]:
    articles = []
    for elem in xml_articles:
        p = parse_pubmed_article(elem)
        if p is None: continue
        articles.append(Article(**p, query_source=pmid_to_query.get(p['pmid'], '')))
    return articles

### 1.8 PMC full-text

### `fetch_pmc_sections`, `_parse_pmc_xml`, `_extract_section_text`, `enrich_with_fulltext`

- `fetch_pmc_sections` — загружает Introduction и Conclusion из PMC full-text. **Вход:** pmc_id (str). **Выход:** tuple[str, str]
- `_parse_pmc_xml` — парсит PMC XML и извлекает секции intro/conclusion. **Вход:** xml_text (str). **Выход:** tuple[str, str]
- `_extract_section_text` — извлекает текст из XML-секции, пропуская формулы и таблицы. **Вход:** Element. **Выход:** str
- `enrich_with_fulltext` — обогащает список Article полнотекстовыми секциями из PMC. **Вход:** list[Article]. **Выход:** None (модифицирует in-place)

In [ ]:
def fetch_pmc_sections(pmc_id: str) -> tuple[str, str]:
    if not pmc_id: return '', ''
    try:
        resp = api_get(EFETCH_URL, {'db': 'pmc', 'id': pmc_id.replace('PMC',''), 'rettype': 'xml'})
    except Exception as e:
        print(f'  [warn] {pmc_id}: {e}')
        return '', ''
    return _parse_pmc_xml(resp.text)

def _parse_pmc_xml(xml_text):
    try: root = ET.fromstring(xml_text)
    except ET.ParseError: return '', ''
    intro, concl = '', ''
    for sec in root.iter('sec'):
        st = (sec.get('sec-type') or '').lower()
        te = sec.find('title')
        t = (te.text or '').lower().strip() if te is not None else ''
        is_i = st in ('intro','introduction') or 'introduction' in t or 'background' in t
        is_c = st in ('conclusion','conclusions') or 'conclusion' in t
        if is_i and not intro: intro = _extract_section_text(sec)
        elif is_c and not concl: concl = _extract_section_text(sec)
    return intro, concl

def _extract_section_text(el):
    skip = {'xref','table-wrap','fig','disp-formula','inline-formula','ext-link'}
    parts = []
    for n in el.iter():
        if n.tag in skip: continue
        if n.text: parts.append(n.text.strip())
        if n.tail: parts.append(n.tail.strip())
    return ' '.join(filter(None, parts))

def enrich_with_fulltext(articles):
    pmc = [a for a in articles if a.pmc_id]
    print(f'PMC full-text: {len(pmc)} из {len(articles)} статей')
    for i, a in enumerate(pmc, 1):
        print(f'  [{i}/{len(pmc)}] {a.pmc_id}', end=' ')
        intro, concl = fetch_pmc_sections(a.pmc_id)
        a.introduction, a.conclusion = intro, concl
        a.full_text_available = bool(intro or concl)
        print('OK' if a.full_text_available else 'нет секций')
        time.sleep(0.4)

### 1.9 Очистка и фильтрация

### `clean_text`, `clean_articles`, `save_articles`

- `clean_text` — очищает текст от inline-ссылок, URL, email и спецсимволов. **Вход:** str. **Выход:** str
- `clean_articles` — фильтрует статьи по длине абстракта (≥100 симв.). **Вход:** list[Article]. **Выход:** list[Article]
- `save_articles` — сохраняет список Article в JSON-файл. **Вход:** list[Article], Path. **Выход:** None

In [ ]:
def clean_text(text):
    if not text: return ''
    text = unicodedata.normalize('NFKC', text)
    text = re.sub(r'\[\s*\d+(?:\s*[,\-\u2013]\s*\d+)*\s*\]', '', text)
    text = re.sub(r"\(\s*[A-Z][a-zA-Z\-']+(?:\s+(?:et\s+al\.?|and\s+[A-Z][a-zA-Z\-']+))?(?:,?\s*\d{4}[a-z]?)\s*\)", '', text)
    text = re.sub(r'https?://\S+', '', text)
    text = re.sub(r'doi:\s*\S+', '', text, flags=re.IGNORECASE)
    text = re.sub(r'\S+@\S+\.\S+', '', text)
    text = re.sub(r"[^\w\s.,;:!?()\-\u2013/'+\u03b1-\u03c9]", ' ', text)
    return re.sub(r'\s+', ' ', text).strip()

def clean_articles(articles):
    cleaned, rm_short = [], 0
    for a in articles:
        a.title = clean_text(a.title)
        a.abstract = clean_text(a.abstract)
        a.introduction = clean_text(a.introduction)
        a.conclusion = clean_text(a.conclusion)
        if len(a.abstract) < 100: rm_short += 1; continue
        cleaned.append(a)
    print(f'Оставлено: {len(cleaned)} | удалено коротких: {rm_short}')
    return cleaned

def save_articles(articles, path):
    path.parent.mkdir(parents=True, exist_ok=True)
    with open(path, 'w', encoding='utf-8') as f:
        json.dump([asdict(a) for a in articles], f, ensure_ascii=False, indent=2)
    print(f'Сохранено: {path} ({len(articles)} статей)')

### 1.10 Запуск сбора данных

In [ ]:
print('=' * 60)
print('  ФАЗА 1: Сбор данных с PubMed')
print('=' * 60)

if OUTPUT_DIR.exists():
    shutil.rmtree(OUTPUT_DIR)
    print(f'Старые данные удалены: {OUTPUT_DIR}/')
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

pmid_to_query = collect_all_pmids()

print('\nЗагрузка метаданных...')
xml_articles = fetch_pubmed_xml(list(pmid_to_query.keys()))
articles = parse_all_articles(xml_articles, pmid_to_query)
print(f'Загружено: {len(articles)} статей')

print('\nPMC full-text...')
enrich_with_fulltext(articles)
save_articles(articles, RAW_JSON)
raw_count = len(articles)

print('\nОчистка...')
articles = clean_articles(articles)
save_articles(articles, CLEAN_JSON)

ft = sum(1 for a in articles if a.full_text_available)
print(f'\nИтого: {raw_count} -> {len(articles)} статей (с full-text: {ft})')

<a id="part2"></a>
# Часть 2. Эксплораторный анализ (EDA)

## Цели анализа

- **Временное покрытие**: из каких лет статьи? Есть ли тренд роста интереса?
- **Объём текстов**: достаточно ли контекста для RAG? Какая доля full-text?
- **Ключевые термины**: какие мишени и механизмы чаще всего упоминаются?
- **Co-occurrence**: какие мишени встречаются совместно — матрица совместной встречаемости
- **Источники**: в каких журналах больше всего релевантных статей?

Результаты EDA помогают настроить параметры чанкинга и оценить покрытие базы знаний.


### 2.1 Загрузка данных

In [ ]:
with open(CLEAN_JSON, 'r', encoding='utf-8') as f:
    df = pd.DataFrame(json.load(f))

def combine_text(row):
    return ' '.join(p for p in [row.get('title',''), row.get('abstract',''),
                                 row.get('introduction',''), row.get('conclusion','')] if p)

df['combined_text'] = df.apply(combine_text, axis=1)
df['abstract_words'] = df['abstract'].apply(lambda x: len(x.split()) if x else 0)
df['combined_words'] = df['combined_text'].apply(lambda x: len(x.split()))
df['has_ft'] = df['full_text_available'].map({True: 'С full-text', False: 'Только абстракт'})

print(f'Статей: {len(df)}')
print(f'  С full-text: {df["full_text_available"].sum()}')
print(f'  Годы: {df["year"].min()} - {df["year"].max()}')
print(f'  Журналов: {df["journal"].nunique()}')
df[['pmid','title','year','journal','full_text_available']].head(10)


### 2.2 Распределение по годам

In [ ]:
year_counts = df['year'].value_counts().sort_index()
fig, ax = plt.subplots(figsize=(12, 5))
bars = ax.bar(year_counts.index, year_counts.values,
              color=sns.color_palette('viridis', len(year_counts)), edgecolor='white')
for b, v in zip(bars, year_counts.values):
    ax.text(b.get_x()+b.get_width()/2, b.get_height()+0.3,
            str(v), ha='center', fontweight='bold', fontsize=10)
ax.set_xlabel('Год'); ax.set_ylabel('Кол-во статей')
ax.set_title('Распределение статей по годам публикации')
plt.xticks(rotation=45); plt.tight_layout(); plt.show()


### 2.3 Длины текстов и доля full-text

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(16, 5))
sns.histplot(df['abstract_words'], bins=30, kde=True, ax=axes[0], color='steelblue')
axes[0].axvline(df['abstract_words'].median(), color='red', ls='--',
                label=f'Медиана: {df["abstract_words"].median():.0f}')
axes[0].set_title('Длина абстрактов (слова)'); axes[0].legend()

sns.boxplot(data=df, x='has_ft', y='combined_words', ax=axes[1], palette='Set2')
axes[1].set_title('Full-text vs абстракт'); axes[1].set_xlabel('')

ft_vals = df['full_text_available'].value_counts()
ft_labels = ['Full-text' if idx else 'Абстракт' for idx in ft_vals.index]
ft_colors = ['#66b3ff' if idx else '#ff9999' for idx in ft_vals.index]
axes[2].pie(ft_vals.values, labels=ft_labels,
            autopct='%1.1f%%', colors=ft_colors, startangle=90)
axes[2].set_title('Доля full-text')
plt.tight_layout(); plt.show()


### 2.4 TF-IDF + Word Cloud

In [ ]:
vectorizer = TfidfVectorizer(
    max_features=200, stop_words='english',
    ngram_range=(1, 2), min_df=3, max_df=0.75,
    token_pattern=r'[a-zA-Z\u03b1-\u03c9][a-zA-Z\u03b1-\u03c9\-]{2,}',
)
tfidf_matrix = vectorizer.fit_transform(df['combined_text'].tolist())
features = vectorizer.get_feature_names_out()
mean_tfidf = np.asarray(tfidf_matrix.mean(axis=0)).flatten()
top_idx = mean_tfidf.argsort()[::-1][:40]
top_terms = {features[i]: mean_tfidf[i] for i in top_idx}

fig, axes = plt.subplots(1, 2, figsize=(18, 8))
t30 = dict(list(top_terms.items())[:30])
sns.barplot(y=list(t30.keys()), x=list(t30.values()), palette='viridis', ax=axes[0])
axes[0].set_title('Топ-30 терминов (TF-IDF)'); axes[0].set_xlabel('Средний TF-IDF')

wc = WordCloud(width=800, height=400, background_color='white', colormap='viridis', max_words=80)
wc.generate_from_frequencies(top_terms)
axes[1].imshow(wc, interpolation='bilinear'); axes[1].axis('off')
axes[1].set_title('Word Cloud (TF-IDF weighted)')
plt.tight_layout(); plt.show()


### 2.5 Топ журналов

In [ ]:
top_j = df['journal'].value_counts().head(15)
fig, ax = plt.subplots(figsize=(10, 6))
sns.barplot(x=top_j.values, y=top_j.index, palette='mako', ax=ax)
ax.set_title('Топ-15 журналов'); ax.set_xlabel('Статей')
plt.tight_layout(); plt.show()


### 2.6 Co-occurrence матрица терапевтических мишеней

Ключевые терапевтические мишени болезни Альцгеймера часто встречаются вместе в одной статье.
Матрица совместной встречаемости (heatmap) и столбчатая диаграмма топ-15 пар показывают,
какие мишени исследуются совместно — это помогает понять структуру данных перед
построением RAG-системы и выявить тематические кластеры.


In [ ]:
top_idx = mean_tfidf.argsort()[::-1][:15]
TARGET_TERMS = [features[i] for i in top_idx]

n = len(TARGET_TERMS)
cooc = np.zeros((n, n), dtype=int)
for _, row in df.iterrows():
    txt = row['combined_text'].lower()
    present = [i for i, t in enumerate(TARGET_TERMS) if t in txt]
    for i in present:
        for j in present:
            if i != j:
                cooc[i, j] += 1

cooc_df = pd.DataFrame(cooc, index=TARGET_TERMS, columns=TARGET_TERMS)

fig, axes = plt.subplots(1, 2, figsize=(18, 7))

sns.heatmap(cooc_df, annot=True, fmt='d', cmap='YlOrRd',
            linewidths=0.5, ax=axes[0])
axes[0].set_title('Co-occurrence мишеней (кол-во статей)')
axes[0].tick_params(axis='x', rotation=45)

pairs = []
for i in range(n):
    for j in range(i+1, n):
        if cooc[i, j] > 0:
            pairs.append((TARGET_TERMS[i], TARGET_TERMS[j], cooc[i, j]))
pairs.sort(key=lambda x: -x[2])
top_pairs = pairs[:15]

pair_labels = [f'{a} + {b}' for a, b, _ in top_pairs]
pair_vals = [v for _, _, v in top_pairs]
sns.barplot(x=pair_vals, y=pair_labels, palette='rocket_r', ax=axes[1])
axes[1].set_title('Топ-15 пар мишеней по частоте совместного упоминания')
axes[1].set_xlabel('Кол-во статей')

plt.tight_layout(); plt.show()

print('\nНаиболее часто исследуемые пары мишеней:')
for a, b, v in top_pairs[:5]:
    print(f'  {a} + {b}: {v} статей')


<a id="part3"></a>
# Часть 3. Векторная база данных

### 3.1 Конфигурация

In [ ]:
CHROMA_DIR = Path('vectordb')
COLLECTION_NAME = 'alzheimer_children'
EMBEDDING_MODEL = 'all-MiniLM-L6-v2'

CHILD_CHUNK_SIZE = 300
CHILD_CHUNK_OVERLAP = 50
MIN_CHUNK_LENGTH = 40

### 3.2 Построение иерархии Parent → Children

Каждая секция статьи становится **parent-документом**. Внутри parent нарезаются мелкие **child-чанки** для индексации.


In [ ]:
with open(CLEAN_JSON, 'r', encoding='utf-8') as f:
    articles_data = json.load(f)

parents = {}

for art in articles_data:
    base_meta = {k: art.get(k, '') for k in ['pmid','title','journal','year','doi']}
    base_meta['authors'] = ', '.join(art.get('authors', [])[:3])
    for section in ['abstract', 'introduction', 'conclusion']:
        text = art.get(section, '')
        if not text or len(text) < MIN_CHUNK_LENGTH:
            continue
        parent_id = f"{art['pmid']}_{section}"
        parents[parent_id] = {
            'text': text,
            'metadata': {**base_meta, 'section': section, 'parent_id': parent_id},
        }

child_splitter = RecursiveCharacterTextSplitter(
    chunk_size=CHILD_CHUNK_SIZE,
    chunk_overlap=CHILD_CHUNK_OVERLAP,
    separators=['. ', '; ', ', ', ' ', ''],
    keep_separator=True,
)

children = []

for pid, parent in parents.items():
    splits = child_splitter.split_text(parent['text'])
    for i, chunk_text in enumerate(splits):
        if len(chunk_text.strip()) < MIN_CHUNK_LENGTH:
            continue
        child_id = hashlib.md5(f"{pid}_{i}".encode()).hexdigest()[:12]
        children.append({
            'id': child_id,
            'text': chunk_text.strip(),
            'parent_id': pid,
            'metadata': {
                **parent['metadata'],
                'parent_id': pid,
                'child_index': i,
                'child_chars': len(chunk_text),
            },
        })

unique_pmids = len(set(p['metadata']['pmid'] for p in parents.values()))
sec_counts = Counter(p['metadata']['section'] for p in parents.values())
child_lens = [len(c['text']) for c in children]
parent_lens = [len(p['text']) for p in parents.values()]

print(f'Иерархия построена:')
print(f'  Статей: {unique_pmids}')
print(f'  Parents (секций): {len(parents)}')
for s, cnt in sorted(sec_counts.items()): print(f'    {s}: {cnt}')
print(f'  Children (чанков): {len(children)}')
print(f'  Parent длина: мин={min(parent_lens)}, медиана={sorted(parent_lens)[len(parent_lens)//2]}, макс={max(parent_lens)}')
print(f'  Child длина:  мин={min(child_lens)}, медиана={sorted(child_lens)[len(child_lens)//2]}, макс={max(child_lens)}')
print(f'  Среднее children/parent: {len(children)/len(parents):.1f}')

### 3.3 Эмбеддинги children

Индексируются **только children** — маленькие чанки для точного поиска. Parents хранятся отдельно в памяти.


In [ ]:
encoder = SentenceTransformer(EMBEDDING_MODEL)
print(f'Модель: {EMBEDDING_MODEL} ({encoder.get_sentence_embedding_dimension()}d)')

child_texts = [c['text'] for c in children]
child_embeddings = encoder.encode(child_texts, batch_size=64,
                                   show_progress_bar=True,
                                   normalize_embeddings=True).tolist()
print(f'Эмбеддинги children: {len(child_embeddings)}')


### 3.4 ChromaDB 

In [ ]:
if CHROMA_DIR.exists():
    shutil.rmtree(CHROMA_DIR)
    print(f'Старая БД удалена: {CHROMA_DIR}/')

CHROMA_DIR.mkdir(parents=True, exist_ok=True)
chroma_client = chromadb.PersistentClient(
    path=str(CHROMA_DIR),
    settings=Settings(anonymized_telemetry=False),
)

collection = chroma_client.get_or_create_collection(
    name=COLLECTION_NAME,
    metadata={
        'hnsw:space': 'cosine',
        'embedding_model': EMBEDDING_MODEL,
        'architecture': 'hierarchical_parent_child',
        'child_chunk_size': CHILD_CHUNK_SIZE,
    },
)

for i in range(0, len(children), 500):
    end = min(i + 500, len(children))
    batch = children[i:end]
    metas = [{k: (v if isinstance(v, (str, int, float, bool)) else str(v))
              for k, v in c['metadata'].items()} for c in batch]
    collection.add(
        ids=[c['id'] for c in batch],
        documents=[c['text'] for c in batch],
        embeddings=child_embeddings[i:end],
        metadatas=metas,
    )

print(f'ChromaDB: {collection.count()} children в "{COLLECTION_NAME}"')
print(f'Parents в памяти: {len(parents)}')

### 3.5 BM25 индекс (children)

### `bm25_tokenize`

Токенизирует текст для BM25: извлекает слова длиной ≥3 символов в нижнем регистре.

**Вход:** `text` (str)  
**Выход:** `list[str]` — список токенов

In [ ]:
def bm25_tokenize(text):
    return re.findall(r'[a-zA-Z\u03b1-\u03c9][a-zA-Z\u03b1-\u03c9\-]{2,}', text.lower())

bm25_corpus = [bm25_tokenize(c['text']) for c in children]
bm25_index = BM25Okapi(bm25_corpus)
print(f'BM25: {len(bm25_corpus)} children')

### 3.6 Функции поиска: child → parent

После нахождения релевантных **children** возвращаем их **parent** (полную секцию).
Дедупликация по `parent_id` гарантирует, что одна секция не повторяется.


### `search_dense`, `search_bm25`, `search_hybrid_children`, `children_to_parents`

- `search_dense` — dense-поиск по ChromaDB (cosine similarity). **Вход:** query (str), top_k (int). **Выход:** list[dict] с id, sim, meta, text
- `search_bm25` — sparse-поиск BM25 по children. **Вход:** query (str), top_k (int). **Выход:** list[dict] с id, score, meta, text
- `search_hybrid_children` — гибридный поиск (dense + BM25) с RRF fusion. **Вход:** query (str), top_k, dense_k, bm25_k, rrf_k. **Выход:** list[dict] ранжированных children
- `children_to_parents` — агрегирует children-результаты в уникальные parent-секции. **Вход:** list[dict] child_results, max_parents. **Выход:** list[dict] parent-чанков с метаданными

In [ ]:
def search_dense(query, top_k=10):
    q_emb = encoder.encode([query], normalize_embeddings=True).tolist()
    res = collection.query(query_embeddings=q_emb, n_results=top_k,
                           include=['documents','metadatas','distances'])
    return [{'id': res['ids'][0][i],
             'sim': 1 - res['distances'][0][i],
             'meta': res['metadatas'][0][i],
             'text': res['documents'][0][i]}
            for i in range(len(res['ids'][0]))]


def search_bm25(query, top_k=10):
    scores = bm25_index.get_scores(bm25_tokenize(query))
    top_idx = scores.argsort()[::-1][:top_k]
    return [{'id': children[i]['id'], 'score': scores[i],
             'meta': children[i]['metadata'], 'text': children[i]['text']}
            for i in top_idx if scores[i] > 0]


def search_hybrid_children(query, top_k=15, dense_k=20, bm25_k=20, rrf_k=60):
    dense_res = search_dense(query, dense_k)
    bm25_res = search_bm25(query, bm25_k)
    rrf, data = {}, {}
    for rank, r in enumerate(dense_res):
        rrf[r['id']] = rrf.get(r['id'], 0) + 1 / (rrf_k + rank + 1)
        data[r['id']] = r
    for rank, r in enumerate(bm25_res):
        rrf[r['id']] = rrf.get(r['id'], 0) + 1 / (rrf_k + rank + 1)
        if r['id'] not in data: data[r['id']] = r
    ranked = sorted(rrf.items(), key=lambda x: x[1], reverse=True)[:top_k]
    return [{'child_id': cid, 'rrf_score': sc, **data[cid]} for cid, sc in ranked]


def children_to_parents(child_results, max_parents=7):
    parent_scores = {}
    parent_children = {}
    for cr in child_results:
        pid = cr['meta'].get('parent_id', '')
        if not pid or pid not in parents:
            continue
        score = cr.get('sim', cr.get('rrf_score', 0.5))
        if pid not in parent_scores or score > parent_scores[pid]:
            parent_scores[pid] = score
        parent_children.setdefault(pid, []).append(cr['text'][:50])

    ranked_pids = sorted(parent_scores.items(), key=lambda x: x[1], reverse=True)[:max_parents]
    results = []
    for pid, best_score in ranked_pids:
        p = parents[pid]
        results.append({
            **p['metadata'],
            'text': p['text'],
            'similarity': round(best_score, 4),
            'n_matching_children': len(parent_children.get(pid, [])),
        })
    return results

### 3.7 Тестовый поиск: children → parents

In [ ]:
test_q = 'BACE1 inhibitors clinical trials Alzheimer'
print(f'Query: "{test_q}"\n')

child_res = search_hybrid_children(test_q, top_k=15)
print(f'Children найдено: {len(child_res)}')
for cr in child_res[:5]:
    print(f'  rrf={cr["rrf_score"]:.4f} | parent={cr["meta"].get("parent_id","")} | {cr["text"][:60]}...')

parent_res = children_to_parents(child_res, max_parents=5)
print(f'\nParents (уникальные секции): {len(parent_res)}')
for j, p in enumerate(parent_res, 1):
    print(f'  [{j}] PMID:{p["pmid"]} | {p["section"]} | sim={p["similarity"]:.3f} | '
          f'children={p["n_matching_children"]} | {len(p["text"])} символов')
    print(f'      {p["text"][:100]}...')


<a id="part4"></a>
# Часть 4. RAG Pipeline

## Архитектура retrieval-augmented generation

```
Вопрос пользователя
       |
  Dense search + BM25  (Hybrid retrieval)
         |
  RRF Fusion (top 5×K кандидатов)
         |
  Cross-Encoder Re-ranking (top K)
         |
  Prompt Building (токены <CONTEXT>, <SOURCE>, <QUESTION>, <INSTRUCTION>)
         |
  Local LLM Generation
         |
  Postprocessing + Citation Extraction
         |
  Метрики: Faithfulness, Coverage, Accuracy, Relevance
```

### Структурные токены

Для того чтобы LLM чётко разделяла **контекст**, **вопрос** и **инструкцию**, мы используем
XML-подобные токены-разделители:

| Токен | Назначение |
|-------|-----------|
| `<CONTEXT>` / `</CONTEXT>` | Обрамляет весь блок найденных источников |
| `<SOURCE pmid="..." section="..." year="...">` / `</SOURCE>` | Каждый отдельный чанк с метаданными |
| `<QUESTION>` / `</QUESTION>` | Вопрос пользователя |
| `<INSTRUCTION>` / `</INSTRUCTION>` | Блок инструкций для модели |

Модель обучена работать с XML-тегами и может надёжно различать, где заканчивается контекст
и начинается вопрос. Это значительно снижает **hallucination** и повышает **faithfulness**.

| Компонент | Подход | Зачем |
|-----------|--------|-------|
| Hybrid search | BM25 + dense + RRF | BM25 ловит точные термины (BACE1, TREM2), dense — семантику |
| Cross-encoder | `ms-marco-MiniLM-L-6-v2` | Переранжирование кандидатов — +20-40% precision@k |
| Structured prompt | XML-токены | Модель различает контекст/вопрос → выше Faithfulness |
| Top-K | Настраиваемый (по умолчанию 5/7) | Баланс между полнотой и шумом |

### 4.1 Выбор модели и настройки

Интерактивный каталог моделей. `trust_remote_code=True` требуется для некоторых архитектур
(например, Qwen использует собственный attention). В продакшене следует проверять
источник модели на HuggingFace перед использованием этого флага.

In [ ]:
MODEL_CATALOG = OrderedDict({
    'TinyLlama/TinyLlama-1.1B-Chat-v1.0': {
        'label': 'TinyLlama 1.1B — самая лёгкая (CPU ~5 GB)',
        'size': '1.1B', 'vram_fp16': '~3 GB', 'vram_4bit': '~1.5 GB',
        'ram_cpu': '~5 GB', 'quality': '2/5', 'speed': '5/5',
        'license': 'Apache 2.0', 'gated': False,
        'notes': 'Для слабых машин. Низкое качество цитирования.',
    },
    'Qwen/Qwen2-1.5B-Instruct': {
        'label': 'Qwen2 1.5B — рекомендуется (CPU ~7 GB)',
        'size': '1.5B', 'vram_fp16': '~4 GB', 'vram_4bit': '~2 GB',
        'ram_cpu': '~7 GB', 'quality': '3/5', 'speed': '5/5',
        'license': 'Apache 2.0', 'gated': False,
        'notes': 'Лучший для CPU. Приемлемое цитирование.',
    },
    'microsoft/phi-2': {
        'label': 'Phi-2 2.7B — reasoning (CPU ~11 GB)',
        'size': '2.7B', 'vram_fp16': '~6 GB', 'vram_4bit': '~3 GB',
        'ram_cpu': '~11 GB', 'quality': '3/5', 'speed': '4/5',
        'license': 'MIT', 'gated': False,
        'notes': 'Нет chat template (plain prompt).',
    },
    'google/gemma-2-2b-it': {
        'label': 'Gemma 2 2B (gated, нужен HF token)',
        'size': '2.6B', 'vram_fp16': '~6 GB', 'vram_4bit': '~3 GB',
        'ram_cpu': '~11 GB', 'quality': '3/5', 'speed': '4/5',
        'license': 'Gemma License', 'gated': True,
        'notes': 'Нужен HF token.',
    },
    'mistralai/Mistral-7B-Instruct-v0.3': {
        'label': 'Mistral 7B — лучший баланс (GPU 8+ GB)',
        'size': '7.2B', 'vram_fp16': '~15 GB', 'vram_4bit': '~5 GB',
        'ram_cpu': '~30 GB', 'quality': '4/5', 'speed': '3/5',
        'license': 'Apache 2.0', 'gated': False,
        'notes': 'Рекомендуется 4-bit на GPU 8+ GB.',
    },
    'Qwen/Qwen2-7B-Instruct': {
        'label': 'Qwen2 7B (GPU 8+ GB)',
        'size': '7.6B', 'vram_fp16': '~16 GB', 'vram_4bit': '~5 GB',
        'ram_cpu': '~32 GB', 'quality': '4/5', 'speed': '3/5',
        'license': 'Apache 2.0', 'gated': False,
        'notes': 'Отличное следование инструкциям.',
    },
    'meta-llama/Meta-Llama-3.1-8B-Instruct': {
        'label': 'Llama 3.1 8B — лучшее качество (gated, GPU)',
        'size': '8.0B', 'vram_fp16': '~17 GB', 'vram_4bit': '~6 GB',
        'ram_cpu': '~34 GB', 'quality': '5/5', 'speed': '2/5',
        'license': 'Llama 3.1 Community', 'gated': True,
        'notes': 'Нужен HF token. Лучшая faithfulness.',
    },
})

model_dropdown = widgets.Dropdown(
    options=[(v['label'], k) for k, v in MODEL_CATALOG.items()],
    value='Qwen/Qwen2-1.5B-Instruct', description='Модель:',
    style={'description_width': '80px'}, layout=widgets.Layout(width='100%'))
quantize_radio = widgets.RadioButtons(
    options=[('FP16/FP32','none'), ('4-bit NF4','4bit'), ('8-bit','8bit')],
    value='none', description='Квант.:',
    style={'description_width': '80px'}, layout=widgets.Layout(width='100%'))
token_input = widgets.Password(value='', placeholder='hf_XXXX (для gated моделей)',
    description='HF Token:', style={'description_width': '80px'},
    layout=widgets.Layout(width='100%'))
topk_slider = widgets.IntSlider(value=3, min=1, max=15, description='Top-K:',
    style={'description_width': '50px'}, layout=widgets.Layout(width='200px'))
max_tokens_slider = widgets.IntSlider(value=512, min=128, max=2048, step=128,
    description='Max tok:', style={'description_width': '60px'},
    layout=widgets.Layout(width='280px'))

model_info = widgets.Output()
def show_info(change=None):
    model_info.clear_output()
    with model_info:
        info = MODEL_CATALOG[model_dropdown.value]
        print(f'  {model_dropdown.value}')
        print(f'  Размер: {info["size"]} | VRAM: {info["vram_4bit"]} (4bit) | CPU RAM: {info["ram_cpu"]}')
        print(f'  Качество: {info["quality"]} | Скорость: {info["speed"]} | {info["notes"]}')
        if info['gated'] and not token_input.value:
            print(f'  [!] Gated модель — введите HF Token!')
model_dropdown.observe(show_info, names='value'); show_info()

display(widgets.HTML('<h3>Настройки LLM</h3>'))
display(model_dropdown); display(model_info)
display(quantize_radio); display(token_input)
display(widgets.HBox([topk_slider, max_tokens_slider]))

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
SIMILARITY_THRESHOLD = 0.25
TEMPERATURE = 0.3
TOP_P = 0.9
REPETITION_PENALTY = 1.12
MAX_CONTEXT_LENGTH = 4096
RERANKER_MODEL = 'cross-encoder/ms-marco-MiniLM-L-6-v2'
print(f'\nDevice: {DEVICE}')

### 4.2 Применение настроек

Фиксируем выбранные пользователем параметры из виджетов.
После выполнения этой ячейки все последующие шаги используют `HF_MODEL`, `TOP_K` и т.д.

In [ ]:
HF_MODEL = model_dropdown.value
QUANTIZE = quantize_radio.value
HF_TOKEN = token_input.value or None
TOP_K = topk_slider.value
MAX_NEW_TOKENS = max_tokens_slider.value

info = MODEL_CATALOG[HF_MODEL]
print(f'Модель: {HF_MODEL} ({info["size"]}) | quant={QUANTIZE} | top_k={TOP_K}')

### 4.3 Cross-encoder Re-ranker

Cross-encoder принимает пару `(query, document)` и выдаёт единый скор релевантности.
В отличие от bi-encoder (SentenceTransformer), cross-encoder моделирует **взаимодействие**
между запросом и документом через full attention, что даёт значительно более точное
ранжирование (+20-40% precision@k).

Используем `ms-marco-MiniLM-L-6-v2` — компактную модель, обученную на MS MARCO.
Она принимает кандидатов из hybrid search и переранжирует их по релевантности.

### `rerank`

Переранжирует кандидатов из hybrid search с помощью cross-encoder (ms-marco-MiniLM).

**Вход:** `query` (str), `chunks_list` (list[dict]) — кандидаты, `top_n` (int)  
**Выход:** `list[dict]` — top_n чанков, отсортированных по rerank_score

In [ ]:
reranker = CrossEncoder(RERANKER_MODEL, max_length=512)
print(f'Re-ranker: {RERANKER_MODEL}')

def rerank(query: str, chunks_list: list, top_n: int = 5) -> list:
    if not chunks_list:
        return []
    pairs = [(query, c['text']) for c in chunks_list]
    scores = reranker.predict(pairs)
    for c, s in zip(chunks_list, scores):
        c['rerank_score'] = float(s)
    return sorted(chunks_list, key=lambda x: x['rerank_score'], reverse=True)[:top_n]

### 4.4 Структурные токены для разделения контекста, вопроса и инструкций

**Проблема**: маленькие LLM (1-3B) часто путают контекст с инструкцией — галлюцинируют
или игнорируют найденные источники.

**Решение**: XML-подобные токены-разделители. Модели (особенно Qwen, Llama, Mistral)
обучены на данных с XML/HTML-разметкой и хорошо распознают структурные теги.

| Токен | Где используется | Зачем |
|-------|-----------------|-------|
| `<CONTEXT>` / `</CONTEXT>` | Оборачивает все найденные чанки | Модель понимает: «всё внутри — это мои источники» |
| `<SOURCE pmid="..." ...>` / `</SOURCE>` | Каждый отдельный чанк | Модель может точно цитировать `[PMID:X]` |
| `<QUESTION>` / `</QUESTION>` | Вопрос пользователя | Чёткая граница: контекст закончился, вот вопрос |
| `<INSTRUCTION>` / `</INSTRUCTION>` | Правила генерации | Модель знает: это не контент, а управляющие указания |

Эти токены **упоминаются в system prompt**, чтобы модель заранее знала формат.

In [ ]:
CONTEXT_OPEN  = "<CONTEXT>"
CONTEXT_CLOSE = "</CONTEXT>"
SOURCE_OPEN   = "<SOURCE"
SOURCE_CLOSE  = "</SOURCE>"
QUESTION_OPEN  = "<QUESTION>"
QUESTION_CLOSE = "</QUESTION>"
INSTRUCTION_OPEN  = "<INSTRUCTION>"
INSTRUCTION_CLOSE = "</INSTRUCTION>"

print("Структурные токены определены:")
print(f"  Контекст:   {CONTEXT_OPEN} ... {CONTEXT_CLOSE}")
print(f"  Источник:   {SOURCE_OPEN} pmid=\"...\" section=\"...\" year=\"...\">{SOURCE_CLOSE}")
print(f"  Вопрос:     {QUESTION_OPEN} ... {QUESTION_CLOSE}")
print(f"  Инструкция: {INSTRUCTION_OPEN} ... {INSTRUCTION_CLOSE}")

### 4.5 System prompt

In [ ]:
SYSTEM_PROMPT = (
    "### ROLE\n"
    "You are a biomedical research assistant specializing in Alzheimer's disease drug target discovery. "
    "Your sole purpose is to answer research questions using ONLY the provided scientific sources.\n\n"
    
    "### INPUT FORMAT\n"
    "- <CONTEXT>...</CONTEXT>: Contains retrieved source chunks. THIS IS YOUR ONLY KNOWLEDGE BASE.\n"
    "- <SOURCE pmid='X' section='Y'>...</SOURCE>: Individual evidence unit with PubMed ID and section label.\n"
    "- <QUESTION>...</QUESTION>: The user's research query.\n\n"
    
    "### MANDATORY RULES\n"
    "1. SOURCE-ONLY: Answer STRICTLY using information present in <SOURCE> chunks. NO external knowledge, NO inference beyond text.\n"
    "2. CITATION FORMAT: Append [PMID:X] IMMEDIATELY AFTER each factual claim, BEFORE punctuation. If PMID missing → [Source:section_id].\n"
    "3. INSUFFICIENT DATA: If <CONTEXT> lacks information to answer → respond EXACTLY: 'Insufficient data in provided sources.'\n"
    "4. LANGUAGE: Match the language of the <QUESTION> exactly.\n"
    "5. CONCISENESS: Max 250 words. NO introductions ('Based on...', 'According to...'). Use cautious academic language: 'reported', 'suggests', 'may indicate'.\n"
    "6. CONFLICT RESOLUTION: If sources contradict → state both: 'Source A reports X [PMID:A], while Source B suggests Y [PMID:B]'.\n"
    "7. NO SPECULATION: Never extrapolate, generalize, or fill gaps. If evidence is partial, state limitations explicitly.\n\n"
    
    "### OUTPUT VALIDATION (self-check before responding)\n"
    "✓ Every claim has a valid [PMID:X] or [Source:section_id]?\n"
    "✓ All PMIDs/section_ids exist in <CONTEXT>?\n"
    "✓ Zero external knowledge used?\n"
    "✓ Response language matches <QUESTION>?\n"
    "✓ Word count ≤250 and no introductory phrases?"
)

### 4.6 Вспомогательная функция: cosine similarity

Используется в MMR (Maximal Marginal Relevance) для оценки разнообразия
выбранных чанков. Реализация на чистом Python без numpy для совместимости.
При получении векторов разной длины выводит предупреждение (`warnings.warn`) и
обрезает до минимальной длины — это сигнал об ошибке выше по pipeline.


### `cosine_similarity`

Вычисляет косинусное сходство двух векторов (чистый Python, без numpy).
Если длины не совпадают — выводит `warnings.warn` и обрезает до минимальной длины.

**Вход:** `vec1` (list[float]), `vec2` (list[float])  
**Выход:** `float` — значение cosine similarity в диапазоне [-1, 1]

In [ ]:
def cosine_similarity(vec1: list, vec2: list):
    if not vec1 or not vec2:
        return 0.0
    if len(vec1) != len(vec2):
        warnings.warn(f'cosine_similarity: vectors have different lengths ({len(vec1)} vs {len(vec2)}), truncating to min length')
        min_len = min(len(vec1), len(vec2))
        vec1, vec2 = vec1[:min_len], vec2[:min_len]
    dot = sum(a * b for a, b in zip(vec1, vec2))
    norm1 = math.sqrt(sum(a * a for a in vec1))
    norm2 = math.sqrt(sum(b * b for b in vec2))
    if norm1 * norm2 == 0:
        return 0.0
    return dot / (norm1 * norm2)

### 4.7 Maximal Marginal Relevance (MMR)

MMR балансирует **релевантность** (score к запросу) и **разнообразие** (непохожесть на уже выбранные).
Параметр `diversity` управляет балансом:
- `diversity=1.0` — чистая релевантность (жадный top-K)
- `diversity=0.5` — баланс (рекомендуется)
- `diversity=0.0` — максимальное разнообразие

Используется как fallback, когда cross-encoder re-ranking отключён.
Перед вызовом MMR функция `retrieve` вычисляет эмбеддинги кандидатов
через `encoder.encode()` и записывает их в поле `embedding` — это необходимо
для корректного расчёта `cosine_similarity` между кандидатами.


### `apply_mmr`

Maximal Marginal Relevance — выбирает разнообразное подмножество чанков, балансируя релевантность и непохожесть.

**Вход:** `candidates` (list[dict]), `query` (str), `k` (int), `diversity` (float 0..1)  
**Выход:** `list[dict]` — k отобранных чанков

In [ ]:
def apply_mmr(candidates: list, query: str, k: int = 5, diversity: float = 0.5) -> list:
    if len(candidates) <= k:
        return candidates

    selected = []
    pool = sorted(candidates, key=lambda x: x.get('score', 0), reverse=True)

    while len(selected) < k and pool:
        if not selected:
            selected.append(pool.pop(0))
        else:
            best_score = -float('inf')
            best_idx = -1
            for i, doc in enumerate(pool):
                rel = doc.get('score', 0)

                if doc.get('embedding') and selected:
                    raw_sim = max(
                        cosine_similarity(doc['embedding'], sel.get('embedding', []))
                        for sel in selected
                    )
                    sim_to_selected = (raw_sim + 1) / 2
                else:
                    sim_to_selected = 0

                mmr_score = diversity * rel - (1 - diversity) * sim_to_selected
                if mmr_score > best_score:
                    best_score = mmr_score
                    best_idx = i

            if best_idx >= 0:
                selected.append(pool.pop(best_idx))

    return selected

### 4.8 Сборка промпта с XML-токенами (`build_prompt`)

Функция собирает финальный промпт для LLM. Структура:

```
[System message]  ← SYSTEM_PROMPT (правила генерации)

[User message]:
<CONTEXT>
  <SOURCE pmid="12345" section="abstract" year="2023">
    Текст чанка 1...
  </SOURCE>
  <SOURCE pmid="67890" section="introduction" year="2022">
    Текст чанка 2...
  </SOURCE>
</CONTEXT>

<QUESTION>Вопрос пользователя</QUESTION>

<INSTRUCTION>
Правила генерации ответа...
</INSTRUCTION>
```

**Логика**:
1. **Бюджет токенов** — вычисляется `available = max_context_tokens - overhead - reserve_tokens`
2. **Валидация PMID** — проверяется, что PMID состоит из цифр; невалидные заменяются на `"none"`
3. **XML-экранирование** — спецсимволы в тексте (`&`, `<`, `>`) экранируются через `_escape_xml`
4. **Лимит токенов** — чанки добавляются последовательно, пока не исчерпан бюджет
5. **Fallback** — если ни один чанк не прошёл, вставляется `"No sources retrieved."`


### `count_tokens`, `_escape_xml`, `build_prompt`

- `count_tokens` — считает количество токенов в тексте. Сначала пытается использовать токенизатор загруженной модели (`tokenizer.encode`); если модель ещё не загружена — использует tiktoken `cl100k_base` как fallback. **Вход:** str. **Выход:** int
- `_escape_xml` — экранирует спецсимволы XML (&, <, >) в тексте чанков. **Вход:** str. **Выход:** str
- `build_prompt` — собирает финальный промпт в формате messages с XML-токенами (CONTEXT, SOURCE, QUESTION, INSTRUCTION). Валидирует PMID, экранирует XML, контролирует бюджет токенов. **Вход:** query (str), chunks_list (list[dict]), max_context_tokens, reserve_tokens, debug. **Выход:** list[dict] — messages [{role, content}, ...]

In [ ]:
TOKEN_ENCODER = tiktoken.get_encoding("cl100k_base")

def count_tokens(text: str) -> int:
    try:
        if 'tokenizer' in globals() and tokenizer is not None:
            return len(tokenizer.encode(text))
    except Exception:
        pass
    return len(TOKEN_ENCODER.encode(text))


def _escape_xml(text: str) -> str:
    return (text
            .replace("&", "&amp;")
            .replace("<", "&lt;")
            .replace(">", "&gt;"))


def build_prompt(
    query: str,
    chunks_list: list[dict],
    max_context_tokens: int = 3000,
    reserve_tokens: int = 500,
    debug: bool = False
) -> list[dict]:
    context_parts: list[str] = []
    used_tokens = 0

    structure_template = "<CONTEXT>\n\n</CONTEXT>\n\n<QUESTION></QUESTION>\n\n<INSTRUCTION>\n\n</INSTRUCTION>"

    instruction = (
        "Answer <QUESTION> using ONLY <CONTEXT>. Rules:\n"
        "1) Cite EVERY claim as '...text [PMID:X].' (BEFORE period). Multiple: [PMID:A][PMID:B].\n"
        "2) If context lacks info on part of question: state '[PMID:N/A] No data on [topic]'.\n"
        "3) NO repetition: cite each source once per unique point.\n"
        "4) Structure: 1-sentence summary → bullet points → brief synthesis.\n"
        "5) Note conflicts: '[PMID:A] says X, [PMID:B] suggests Y'.\n"
        "6) Answer language = question language. Keep PMIDs/technical terms unchanged.\n\n"
        "<ANSWER>:"
    )
    
    overhead = count_tokens(structure_template + "\n\n" + instruction_template)
    available = max_context_tokens - overhead - reserve_tokens

    for chunk in chunks_list:
        pmid = chunk.get("pmid") or chunk.get("meta", {}).get("pmid") or "none"
        section = chunk.get("section") or chunk.get("meta", {}).get("section") or "unknown"
        year = chunk.get("year") or chunk.get("meta", {}).get("year") or ""
        text = str(chunk.get("text", "")).strip()

        if not text:
            continue

        pmid_str = str(pmid).strip()
        if pmid_str and pmid_str.lower() not in ("none", "unknown", "null", ""):
            if pmid_str.isdigit():
                pmid_clean = pmid_str
            else:
                if debug:
                    print(f"Invalid PMID '{pmid_str}' -> replaced with 'none'")
                pmid_clean = "none"
        else:
            pmid_clean = "none"

        text_escaped = _escape_xml(text)

        year_str = str(year).strip() if year else ""
        source_tag = f'<SOURCE pmid="{pmid_clean}" section="{section}" year="{year_str}">'
        source_block = f"{source_tag}{text_escaped}</SOURCE>"

        chunk_tokens = count_tokens(source_block) + 10
        if used_tokens + chunk_tokens > available:
            if debug:
                print(f"Context limit reached after {len(context_parts)} chunks")
            break

        context_parts.append(source_block)
        used_tokens += chunk_tokens

    if context_parts:
        context_block = "\n".join(context_parts)
    else:
        context_block = '<SOURCE pmid="none" section="none" year="">No sources retrieved.</SOURCE>'

    user_content = (
        f"<CONTEXT>\n{context_block}\n</CONTEXT>\n\n"
        f"<QUESTION>{query}</QUESTION>\n\n"
        f"<INSTRUCTION>{instruction}</INSTRUCTION>"
    )

    if debug:
        cited_in_context = re.findall(r'pmid="([^"]+)"', context_block)
        print(f"DEBUG: PMIDs in context: {set(cited_in_context)}")
        print(f"DEBUG: Context tokens: {used_tokens}/{available}")

    return [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": user_content},
    ]

### Пример: генерация контекста и промпта

Демонстрация того, как `retrieve` формирует контекст, а `build_prompt` собирает финальный промпт для LLM.

In [ ]:
example_query = "What role does TREM2 play as a therapeutic target in Alzheimer's disease?"

print("=" * 60)
print("  ПРИМЕР: Генерация контекста и промпта")
print("=" * 60)

print(f"\nЗапрос: {example_query}")

print("\n--- Шаг 1: Hybrid search (children) ---")
child_res = search_hybrid_children(example_query, top_k=10)
print(f"Найдено children: {len(child_res)}")
for cr in child_res[:3]:
    print(f"  rrf={cr['rrf_score']:.4f} | {cr['text'][:80]}...")

print("\n--- Шаг 2: Агрегация children → parents ---")
parent_res = children_to_parents(child_res, max_parents=5)
print(f"Уникальных parents: {len(parent_res)}")
for j, p in enumerate(parent_res, 1):
    print(f"  [{j}] PMID:{p['pmid']} | {p['section']} | sim={p['similarity']:.3f} | {len(p['text'])} симв.")

print("\n--- Шаг 3: Сборка промпта (build_prompt) ---")
messages = build_prompt(example_query, parent_res, debug=True)

print(f"\nСообщений в промпте: {len(messages)}")
print(f"  system: {len(messages[0]['content'])} символов")
print(f"  user:   {len(messages[1]['content'])} символов")

print("\n--- System prompt ---")
print(messages[0]['content'])

print("\n--- User content ---")
print(messages[1]['content'])
print("...")

### 4.9 Основная функция retrieval (`retrieve`)

Оркестрирует весь pipeline поиска:

1. **Hybrid search** по children (dense + BM25 + RRF fusion) или только dense
2. **Child → Parent** — агрегация мелких чанков в полные секции
3. **Фильтрация** по порогу similarity (SIMILARITY_THRESHOLD = 0.3)
4. **Нормализация** scores в диапазон [0, 1] (min-max)
5. **Fallback** — если после фильтрации не осталось кандидатов, берутся top-3 child_results с извлечением метаданных из `meta`
6. **Re-ranking** (cross-encoder) или **MMR** с предварительным вычислением эмбеддингов кандидатов
7. **Валидация** метаданных — проверка обязательных полей (pmid, title, journal, year, section, text), приведение year к int

Параметры `use_hybrid` и `use_rerank` управляются из интерфейса.

### `retrieve`

Оркестрирует полный pipeline поиска: hybrid search → parent aggregation → фильтрация → нормализация → fallback при пустых кандидатах → re-ranking/MMR (с вычислением эмбеддингов) → валидация метаданных.

**Вход:** `query` (str), `top_k` (int), `use_hybrid` (bool), `use_rerank` (bool)  
**Выход:** `list[dict]` — top_k чанков с метаданными, готовых для build_prompt

In [ ]:
def retrieve(query: str, top_k: int = 5, use_hybrid: bool = True, use_rerank: bool = True) -> list:
    SIMILARITY_THRESHOLD = 0.3
    children_k = top_k * 5

    if use_hybrid:
        child_results = search_hybrid_children(
            query, top_k=children_k, dense_k=children_k, bm25_k=children_k
        )
    else:
        dense = search_dense(query, top_k=children_k)
        child_results = [
            {'child_id': d['id'], 'score': d.get('sim', d.get('rrf_score', 0.0)), **d}
            for d in dense
        ]

    candidates = children_to_parents(child_results, max_parents=top_k * 3)

    for c in candidates:
        c['score'] = c.get('score', c.get('similarity', c.get('rrf_score', 0.0)))

    candidates = [c for c in candidates if c.get('score', 0.0) >= SIMILARITY_THRESHOLD]

    if candidates:
        scores = [c['score'] for c in candidates]
        min_s, max_s = min(scores), max(scores)
        if max_s > min_s:
            for c in candidates:
                c['score'] = (c['score'] - min_s) / (max_s - min_s)

    if not candidates:
        fallback = sorted(child_results, key=lambda x: x.get('score', x.get('rrf_score', 0)), reverse=True)[:3]
        for fb in fallback:
            meta = fb.get('meta', {})
            candidates.append({
                'pmid': meta.get('pmid', ''),
                'title': meta.get('title', ''),
                'journal': meta.get('journal', ''),
                'year': meta.get('year', ''),
                'section': meta.get('section', ''),
                'text': fb.get('text', ''),
                'similarity': fb.get('score', fb.get('rrf_score', 0)),
                'score': fb.get('score', fb.get('rrf_score', 0)),
            })
        if not candidates:
            return []

    if use_rerank and len(candidates) > 1:
        candidates = rerank(query, candidates, top_n=top_k)
    else:
        cand_texts = [c.get('text', '') for c in candidates]
        if cand_texts:
            cand_embs = encoder.encode(cand_texts, normalize_embeddings=True).tolist()
            for c, emb in zip(candidates, cand_embs):
                c['embedding'] = emb
        candidates = apply_mmr(candidates, query, k=top_k, diversity=0.5)

    cleaned = []
    required_fields = ['pmid', 'title', 'journal', 'year', 'section', 'text']

    for c in candidates[:top_k]:
        text = c.get('text', '')
        if not text or len(str(text).strip()) < 20:
            continue

        y = c.get('year')
        if y:
            try:
                c['year'] = int(y)
            except (ValueError, TypeError):
                c['year'] = 0
        else:
            c['year'] = 0

        for field in required_fields:
            if field == 'year':
                continue
            if field not in c or not c[field]:
                c[field] = None if field == 'pmid' else 'Unknown'

        if c.get('pmid') and not str(c['pmid']).isdigit():
            c['pmid'] = None

        cleaned.append(c)

    return cleaned

### 4.10 Загрузка LLM

`trust_remote_code=True` необходим для архитектур Qwen и некоторых других моделей,
которые регистрируют собственные классы. Используйте только модели из проверенных
источников (официальные организации на HuggingFace).

Поддерживается:
- **FP16** — для GPU с достаточной VRAM
- **4-bit NF4** — квантизация через bitsandbytes (GPU only)
- **8-bit** — промежуточный вариант (GPU only)
- **FP32** — для CPU

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

tk = {'trust_remote_code': True}
lk = {'trust_remote_code': True}
if HF_TOKEN:
    tk['token'] = HF_TOKEN
    lk['token'] = HF_TOKEN

print(f'Загрузка: {HF_MODEL}...')
tokenizer = AutoTokenizer.from_pretrained(HF_MODEL, **tk)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

if QUANTIZE == '4bit' and DEVICE == 'cuda':
    from transformers import BitsAndBytesConfig
    lk['quantization_config'] = BitsAndBytesConfig(
        load_in_4bit=True, bnb_4bit_compute_dtype=torch.float16,
        bnb_4bit_quant_type='nf4', bnb_4bit_use_double_quant=True)
    lk['device_map'] = 'auto'
elif QUANTIZE == '8bit' and DEVICE == 'cuda':
    from transformers import BitsAndBytesConfig
    lk['quantization_config'] = BitsAndBytesConfig(load_in_8bit=True)
    lk['device_map'] = 'auto'
elif DEVICE == 'cuda':
    lk['torch_dtype'] = torch.float16
    lk['device_map'] = 'auto'
else:
    lk['torch_dtype'] = torch.float32

llm_model = AutoModelForCausalLM.from_pretrained(HF_MODEL, **lk)
if DEVICE == 'cpu' and QUANTIZE == 'none':
    llm_model = llm_model.to('cpu')
llm_model.eval()

has_chat = hasattr(tokenizer, 'chat_template') and tokenizer.chat_template is not None
param_count = sum(p.numel() for p in llm_model.parameters()) / 1e9
print(f'Загружено: {HF_MODEL} ({param_count:.2f}B params, chat_template={has_chat})')

### 4.11 Генерация ответа (`generate_answer`)

Формирует промпт через `build_prompt`, передаёт в LLM и выполняет постобработку:

1. **Chat template** — если модель поддерживает (Qwen, Llama, Mistral), используем `apply_chat_template`. Результат может быть `Tensor` или `dict` (зависит от версии transformers) — обрабатываются оба варианта через `isinstance` проверку
2. **Plain prompt** — для моделей без chat template (Phi-2) используем формат `### System / ### User / ### Answer`
3. **Условный sampling** — если `TEMPERATURE > 0`, используется `do_sample=True` с `temperature` и `top_p`; иначе — greedy decoding (`do_sample=False`)
4. **Постобработка**:
   - Удаление случайно повторённых системных маркеров
   - Обнаружение и обрезка зацикливания (>2 одинаковых строки подряд)
   - Удаление XML-токенов из ответа (модель иногда копирует их)

### `generate_answer`

Генерирует ответ LLM по найденным чанкам: формирует промпт, обрабатывает результат `apply_chat_template` (Tensor или dict), запускает модель с условным sampling (greedy при TEMPERATURE=0, stochastic при TEMPERATURE>0) и постобрабатывает результат.

**Вход:** `query` (str), `chunks_list` (list[dict]), `max_new_tokens` (int)  
**Выход:** `str` — текст ответа с цитатами [PMID:X]

In [ ]:
def generate_answer(query: str, chunks_list: list,
                    max_new_tokens: int = MAX_NEW_TOKENS) -> str:
    msgs = build_prompt(query, chunks_list)
    # print(f"DEBUG MSGS: {msgs}") 
    if has_chat:
        chat_out = tokenizer.apply_chat_template(
            msgs, tokenize=True, return_tensors="pt", add_generation_prompt=True
        )
        if isinstance(chat_out, torch.Tensor):
            inputs = {"input_ids": chat_out}
        else:
            inputs = chat_out
    else:
        prompt = (
            f"### System:\n{msgs[0]['content']}\n\n"
            f"### User:\n{msgs[1]['content']}\n\n### Answer:\n"
        )
        inputs = tokenizer(prompt, return_tensors="pt")

    if DEVICE == "cuda":
        inputs = {k: v.to("cuda") for k, v in inputs.items()}

    ilen = inputs["input_ids"].shape[1]
    max_gen = min(max_new_tokens, tokenizer.model_max_length - ilen - 8)

    if max_gen <= 0:
        return "Insufficient data in provided sources."

    with torch.no_grad():
        gen_kwargs = dict(
            **inputs,
            max_new_tokens=max_gen,
            repetition_penalty=REPETITION_PENALTY,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id,
        )
        if TEMPERATURE > 0:
            gen_kwargs.update(do_sample=True, temperature=TEMPERATURE, top_p=TOP_P)
        else:
            gen_kwargs.update(do_sample=False)
        out = llm_model.generate(**gen_kwargs)

    ans = tokenizer.decode(out[0][ilen:], skip_special_tokens=True).strip()

    for marker in ["### System:", "CONTEXT:", "QUESTION:", "<CONTEXT>", "</CONTEXT>",
                    "<SOURCE", "</SOURCE>", "<QUESTION>", "</QUESTION>",
                    "<INSTRUCTION>", "</INSTRUCTION>"]:
        if ans.startswith(marker):
            ans = ans[len(marker):].strip()

    clean = []
    prev = None
    repeat_count = 0
    for line in ans.splitlines():
        if line == prev:
            repeat_count += 1
            if repeat_count > 2:
                break
        else:
            repeat_count = 0
        clean.append(line)
        prev = line

    result = "\n".join(clean).strip()
    for tag in [CONTEXT_OPEN, CONTEXT_CLOSE, SOURCE_CLOSE,
                QUESTION_OPEN, QUESTION_CLOSE,
                INSTRUCTION_OPEN, INSTRUCTION_CLOSE]:
        result = result.replace(tag, "")
    result = re.sub(r'<SOURCE[^>]*>', '', result)

    return result.strip()

### 4.12 Proxy-метрики качества (`compute_metrics`)

Оценка качества ответа без human evaluation. Все метрики вычисляются автоматически:

| Метрика | Что измеряет | Формула |
|---------|-------------|---------|
| **Faithfulness** | Доля предложений с цитатой `[PMID:X]` | cited_sentences / total_sentences |
| **Source Coverage** | Доля PMID из retrieval, процитированных в ответе | cited_pmids ∩ source_pmids / source_pmids |
| **Citation Accuracy** | Доля цитат, ссылающихся на реальные источники | cited_pmids ∩ source_pmids / cited_pmids |
| **Context Similarity** | Средний retrieval score чанков (cosine sim или RRF) | mean(chunk_similarity) |
| **Answer Relevance** | Cosine similarity между эмбеддингами вопроса и ответа | cos_sim(embed(query), embed(answer)) |

### `compute_metrics`

Вычисляет proxy-метрики качества ответа: faithfulness, source coverage, citation accuracy, context similarity, answer relevance.

**Вход:** `query` (str), `answer` (str), `chunks_list` (list[dict])  
**Выход:** `dict` — словарь метрик

In [ ]:
def compute_metrics(query: str, answer: str, chunks_list: list) -> dict:
    m = {}

    sentences = [s.strip() for s in re.split(r'[.!?]+', answer) if len(s.strip()) > 20]
    cited_sentences = sum(1 for s in sentences if re.search(r'\[PMID:\d+\]', s))
    m['faithfulness'] = round(cited_sentences / max(len(sentences), 1), 3)

    source_pmids = set(str(c.get('pmid', '')) for c in chunks_list if c.get('pmid'))
    cited_pmids  = set(re.findall(r'PMID:(\d+)', answer))
    m['source_coverage'] = round(
        len(cited_pmids & source_pmids) / max(len(source_pmids), 1), 3
    )

    m['citation_accuracy'] = round(
        len(cited_pmids & source_pmids) / max(len(cited_pmids), 1), 3
    )

    sims = [c.get('similarity', 0) for c in chunks_list if c.get('similarity')]
    m['avg_context_sim'] = round(np.mean(sims), 3) if sims else 0.0

    embs = encoder.encode([query, answer], normalize_embeddings=True)
    m['answer_relevance'] = round(float(embs[0] @ embs[1]), 3)

    m['answer_words'] = len(answer.split())

    return m

### 4.13 Пример: полный pipeline retrieve → build_prompt → generate_answer

Демонстрация полного цикла от вопроса до сгенерированного ответа с метриками.

In [ ]:
demo_query = "What are potential targets for Alzheimer's disease treatment?"

print("=" * 60)
print("  ПРИМЕР: Полный RAG pipeline")
print("=" * 60)

print(f"\nЗапрос: {demo_query}")

print("\n--- Retrieve ---")
demo_chunks = retrieve(demo_query, top_k=5, use_hybrid=True, use_rerank=True)
print(f"Найдено чанков: {len(demo_chunks)}")
for i, c in enumerate(demo_chunks, 1):
    print(f"  [{i}] PMID:{c.get('pmid','N/A')} | {c.get('section','?')} | "
          f"score={c.get('rerank_score', c.get('similarity', 0)):.3f} | {len(c.get('text',''))} симв.")

print("\n--- Build Prompt ---")
demo_messages = build_prompt(demo_query, demo_chunks, debug=True)
print(f"\nSystem prompt: {len(demo_messages[0]['content'])} символов")
print(f"User content:  {len(demo_messages[1]['content'])} символов")
print(f"Всего токенов: ~{count_tokens(demo_messages[0]['content']) + count_tokens(demo_messages[1]['content'])}")

print("\n--- user content ---")
print(demo_messages[1]['content'])
print("...")

print("\n--- Generate Answer ---")
demo_answer = generate_answer(demo_query, demo_chunks)
print(demo_answer)

print("\n--- Metrics ---")
demo_metrics = compute_metrics(demo_query, demo_answer, demo_chunks)
for k, v in demo_metrics.items():
    print(f"  {k}: {v}")

### 4.14 Прогон тестовых вопросов

Три обязательных вопроса по заданию:
1. **Targets** — какие терапевтические мишени описаны в литературе?
2. **Druggability** — доступны ли мишени для малых молекул, биологиков или иных модальностей?
3. **Research gaps** — какие дополнительные исследования нужны?

Для каждого вопроса замеряется время retrieval и generation, вычисляются proxy-метрики.

In [ ]:
TEST_QUESTIONS = [
    "What are potential targets for Alzheimer's disease treatment?",
    'Are the targets druggable with small molecules, biologics, or other modalities?',
    'What additional studies are needed to advance these targets?',
]

all_results = []
for i, q in enumerate(TEST_QUESTIONS, 1):
    print(f'\n{"-"*60}')
    print(f'  Q{i}: {q}')
    print('-' * 60)

    t0 = time.time()
    found = retrieve(q, top_k=TOP_K, use_hybrid=True, use_rerank=True)
    ret_ms = (time.time() - t0) * 1000

    t1 = time.time()
    answer = generate_answer(q, found)
    gen_ms = (time.time() - t1) * 1000

    m = compute_metrics(q, answer, found)
    m['retrieval_ms'] = round(ret_ms, 1)
    m['generation_ms'] = round(gen_ms, 1)

    all_results.append({
        'query': q, 'answer': answer, 'sources': found, 'metrics': m
    })

    print(f'\nОтвет: {answer}')
    print(f'faith={m["faithfulness"]:.2f} cov={m["source_coverage"]:.2f} '
          f'rel={m["answer_relevance"]:.2f} | {ret_ms:.0f}+{gen_ms:.0f}ms')

    gc.collect()
    if DEVICE == 'cuda':
        torch.cuda.empty_cache()

### 4.15 Сводка proxy-метрик

Агрегированные метрики по всем тестовым вопросам с цветовой индикацией
и интерпретацией результатов.

In [ ]:
mlist = [r['metrics'] for r in all_results]
mdf = pd.DataFrame(mlist)
cols = ['faithfulness','source_coverage','citation_accuracy','avg_context_sim',
        'answer_relevance','answer_words','retrieval_ms','generation_ms']
summary = mdf[cols].describe().loc[['mean','min','max']].T

display(Markdown('## Сводка метрик'))
display(summary.style.format('{:.3f}').background_gradient(cmap='RdYlGn', axis=None))

checks = [
    ('faithfulness', 0.7, 0.4, 'Faithfulness',
     'хорошо цитирует', 'часть без цитат — нужна более крупная модель', 'нужна 7B+'),
    ('source_coverage', 0.6, 0.3, 'Source coverage',
     'хорошее покрытие', 'умеренное — увеличьте top_k', 'игнорирует источники'),
    ('avg_context_sim', 0.4, 0.3, 'Context relevance',
     'релевантные чанки', 'рассмотрите PubMedBERT', 'пересмотрите чанкинг'),
    ('answer_relevance', 0.6, 0.4, 'Answer relevance',
     'ответы по теме', 'умеренная', 'ответы не по теме'),
]
print('\nВЫВОДЫ:')
for key, good, ok, name, g, m_, b in checks:
    val = mdf[key].mean()
    if val >= good:   print(f'  [OK] {name}: {g} ({val:.3f})')
    elif val >= ok:   print(f'  [~~] {name}: {m_} ({val:.3f})')
    else:             print(f'  [!!] {name}: {b} ({val:.3f})')
print(f'\n  Модель: {HF_MODEL} | Hybrid+Rerank | Top-K={TOP_K}')

<a id="part5"></a>
# Часть 5. Интерактивный интерфейс

## Функциональность

Интерфейс на `ipywidgets` позволяет исследователю:

1. **Ввести произвольный вопрос** или выбрать из примеров
2. **Настроить параметры**: Top-K, max tokens, hybrid search, re-ranking
3. **Получить ответ** с кликабельными ссылками на PubMed `[PMID:XXXXX]`
4. **Просмотреть источники** с оценками релевантности и ссылками
5. **Оценить качество** — proxy-метрики для каждого ответа


In [ ]:

query_input = widgets.Textarea(
    value="What are potential targets for Alzheimer's disease treatment?",
    placeholder='Введите вопрос на английском...',
    description='Вопрос:',
    layout=widgets.Layout(width='100%', height='80px'),
    style={'description_width': '80px'}
)

examples_dd = widgets.Dropdown(
    options=[
        '--- Выберите пример ---',
        "What are potential targets for Alzheimer's disease treatment?",
        'Are the targets druggable with small molecules, biologics, or other modalities?',
        'What additional studies are needed to advance these targets?',
    ],
    description='Пример:',
    style={'description_width': '60px'},
    layout=widgets.Layout(width='100%')
)

ui_topk = widgets.IntSlider(
    value=7, min=1, max=15,
    description='Top-K:',
    style={'description_width': '50px'},
    layout=widgets.Layout(width='170px')
)

ui_maxtok = widgets.IntSlider(
    value=512, min=128, max=2048, step=128,
    description='Tokens:',
    style={'description_width': '55px'},
    layout=widgets.Layout(width='220px')
)

hybrid_tog = widgets.ToggleButton(
    value=True, description='Hybrid search',
    button_style='info',
    layout=widgets.Layout(width='130px')
)

rerank_tog = widgets.ToggleButton(
    value=True, description='Re-ranking',
    button_style='info',
    layout=widgets.Layout(width='120px')
)

search_btn = widgets.Button(
    description='Найти',
    button_style='primary',
    layout=widgets.Layout(width='120px', height='38px')
)

out = widgets.Output(
    layout=widgets.Layout(
        border='1px solid #ccc',
        padding='12px',
        width='100%'
    )
)


def on_example_select(change):
    if change['new'] != '--- Выберите пример ---':
        query_input.value = change['new']

examples_dd.observe(on_example_select, names='value')


def on_search(b):
    out.clear_output()
    with out:
        q = query_input.value.strip()
        if not q:
            display(Markdown('Введите вопрос.'))
            return

        display(Markdown(f'## Вопрос\n{q}'))

        t0 = time.time()
        found = retrieve(
            q,
            top_k=ui_topk.value,
            use_hybrid=hybrid_tog.value,
            use_rerank=rerank_tog.value
        )
        ret_ms = (time.time() - t0) * 1000

        if not found:
            display(Markdown('Ничего не найдено. Попробуйте переформулировать запрос.'))
            return

        display(Markdown(f'Найдено источников: **{len(found)}** ({ret_ms:.0f} мс)'))
        display(Markdown('*Генерация ответа...*'))

        t1 = time.time()
        answer = generate_answer(
            q,
            found,
            max_new_tokens=ui_maxtok.value
        )
        gen_ms = (time.time() - t1) * 1000

        display(Markdown('---\n## Ответ'))
        display(Markdown(answer))

        display(Markdown(
            f'*Время: {ret_ms:.0f} мс (retrieval) + '
            f'{gen_ms:.0f} мс (generation) | `{HF_MODEL}`*'
        ))

        display(Markdown('---\n## Источники'))

        for j, c in enumerate(found, 1):
            sim = c.get('similarity', 0.0)
            rr = c.get('rerank_score', '')
            score_str = f'sim={sim:.3f}'
            if rr != '':
                score_str += f', rerank={rr:.2f}'

            display(Markdown(
                f'**[{j}]** {c.get("section","")} '
                f'PMID:{c.get("pmid","")} '
                f'({score_str}, {c.get("year","")})'
            ))

            display(Markdown(f'> {c["text"][:200]}...'))

        display(Markdown('---\n## Метрики качества ответа'))
        m = compute_metrics(q, answer, found)

        for k, v in m.items():
            if isinstance(v, float):
                status = (
                    'хорошо' if v >= 0.6 else
                    'умеренно' if v >= 0.3 else
                    'низко'
                )
                display(Markdown(f'- **{k}**: `{v:.3f}` ({status})'))
            else:
                display(Markdown(f'- **{k}**: `{v}`'))

        gc.collect()
        if DEVICE == 'cuda':
            torch.cuda.empty_cache()


search_btn.on_click(on_search)


display(widgets.HTML('<h2>Alzheimer Target Discovery — RAG Agent</h2>'))
display(examples_dd)
display(query_input)
display(widgets.HBox([
    ui_topk,
    ui_maxtok,
    hybrid_tog,
    rerank_tog,
    search_btn
]))
display(out)

# Теоретические вопросы

---

## 1. На какие модальности данных можно расширить решение?

Текущий pipeline работает только с текстом (абстракты, введения, заключения из PubMed/PMC). Расширение возможно на следующие модальности:

**Табличные данные** — таблицы из статей содержат результаты экспериментов: IC50, EC50, Ki значения для ингибиторов, результаты клинических испытаний (p-value, hazard ratio), данные экспрессии генов. Это структурированная информация, которую текущий текстовый RAG теряет при парсинге XML.

**Изображения и графики** — гистологические срезы мозга (иммуногистохимия амилоидных бляшек и тау-клубков), графики dose-response, heatmap'ы экспрессии, микрофотографии конфокальной микроскопии. Визуальные данные часто несут информацию, отсутствующую в тексте.

**Молекулярные структуры** — 3D-структуры белков-мишеней (PDB), SMILES/InChI представления малых молекул, данные о белок-лигандных взаимодействиях. Критично для вопросов о druggability мишеней.

**Геномные и омиксные данные** — последовательности ДНК/РНК, данные GWAS (SNP, ассоциации), протеомика, метаболомика. Хранятся в специализированных форматах (FASTA, VCF, GEO matrix).

**Графы знаний** — сети белок-белковых взаимодействий (PPI), сигнальные пути (KEGG, Reactome), онтологии (Gene Ontology). Структура «сущность—связь—сущность» плохо ложится на текущий векторный поиск по тексту в ChromaDB.

---

## 2. Как это можно сделать?

### Табличные данные

Парсинг таблиц из XML/HTML статей (тег `<table-wrap>` в PMC — аналогично тому, как в текущем pipeline парсятся `<sec>` через `_parse_pmc_xml`). Каждая таблица сериализуется в текст двумя способами: linearization (строка за строкой) для индексации в ChromaDB наравне с текущими child-чанками, и сохранение в структурированном виде (DataFrame) для точных числовых запросов. На этапе retrieval добавляется отдельный Table Retriever — по запросу «IC50 BACE1 inhibitors» он возвращает релевантные строки таблиц. Результаты объединяются с текстовым поиском через RRF (аналогично текущей `search_hybrid_children`). В промпт таблицы вставляются в Markdown-формате внутри отдельного тега `<TABLE_SOURCE>` — расширение текущей XML-разметки `<SOURCE>`.

### Изображения и графики

Используется vision-language модель (LLaVA, Qwen-VL, GPT-4V через API) для генерации текстовых описаний (caption) каждого рисунка. Полученные описания индексируются как обычные текстовые child-чанки с метаданными `section=figure` и связываются с parent-документом — расширение текущей иерархии parent-child. При retrieval чанки с описаниями рисунков возвращаются наравне с текстовыми и проходят тот же re-ranking через cross-encoder. Для мультимодальной генерации ответа изображение передаётся в VLM вместе с текстовым контекстом.

### Молекулярные структуры

Интеграция специализированных эмбеддингов: MolBERT / ChemBERTa для SMILES, ESM-2 для белковых последовательностей. Создаётся отдельная коллекция в ChromaDB с молекулярными эмбеддингами (аналогично текущей коллекции `alzheimer_children`, но с другим пространством). Поиск по запросу «small molecule inhibitor of GSK3β» идёт параллельно: текстовый retrieval находит статьи через текущий hybrid search, молекулярный retrieval — конкретные соединения из баз (ChEMBL, PDB). Результаты объединяются через RRF и подаются в промпт.

### Геномные данные

Подключение внешних API (NCBI Gene, UniProt, STRING) как tool-use для агента — расширение текущего `api_get` хелпера с retry-логикой. Агент сам решает, когда нужно запросить данные об экспрессии гена или PPI-сети. Результаты API-вызовов форматируются в текст и добавляются в `<CONTEXT>` блок промпта. Для GWAS-данных — предварительная индексация ключевых ассоциаций (ген, SNP, p-value, популяция) как структурированных чанков в ChromaDB.

### Графы знаний

Knowledge Graph Embedding (TransE, RotatE) для сетей взаимодействий. При запросе «upstream regulators of tau phosphorylation» граф обходится от узла TAU по рёбрам phosphorylation/regulation, извлекаются связанные сущности (GSK3β, CDK5, PP2A). Эти сущности используются для query expansion — обогащения поискового запроса перед текущим hybrid search (BM25 + dense). Альтернативно — SPARQL-запросы к Wikidata/UniProt RDF как tool-use.

### Общая архитектура мультимодального RAG

Расширение текущего pipeline:

```
Запрос пользователя
       |
  Query Router (классификатор типа запроса)
       |
  ┌────┴────┬──────────┬──────────┐
  Text    Table    Molecular    KG
  Retriever Retriever Retriever  Traversal
  (текущий  (новый)  (ChromaDB   (graph
  hybrid)           коллекция)  embedding)
  │         │         │          │
  └────┬────┴─────────┴──────────┘
       |
  Cross-modal RRF Fusion
       |
  Cross-Encoder Re-ranking (текущий ms-marco-MiniLM)
       |
  Multimodal Prompt Assembly (расширение build_prompt)
       |
  VLM / LLM Generation
```

---

## 3. Какие модели и почему выбраны для решения

### Embedding: `all-MiniLM-L6-v2`

Даёт приемлемое качество при минимальных ресурсах и не требует доменной адаптации.

### Re-ranker: `cross-encoder/ms-marco-MiniLM-L-6-v2`

Cross-encoder для финального ранжирования: он принимает пару `(query, document)` целиком через full attention и выдаёт единый скор релевантности. В отличие от bi-encoder, который кодирует запрос и документ независимо, cross-encoder моделирует взаимодействие между ними, что даёт +20–40% precision@k. Обучен на MS MARCO (530К query-passage пар). В pipeline вызывается после hybrid search — переранжирует кандидатов с `max_length=512` токенов. Компактный размер (6 слоёв) позволяет использовать на CPU.

### Sparse retrieval: BM25 (Okapi)

Реализация `BM25Okapi` из `rank-bm25`. Подключается как «страховка» от семантических ошибок dense-поиска: ищет по точным вхождениям токенов, что критично для различения похожих терминов (BACE1/BACE2, GSK3α/GSK3β), которые нейросеть может спутать в эмбеддинг-пространстве. Токенизация — regex-паттерн `[a-zA-Zα-ω][a-zA-Zα-ω\-]{2,}` (слова ≥3 символов, включая греческие буквы для биохимических терминов). Индекс строится по всем child-чанкам (тот же корпус, что и ChromaDB).

### Fusion: Reciprocal Rank Fusion (RRF)

Объединение результатов dense (ChromaDB) и sparse (BM25) поиска. Формула: `score(d) = Σ 1/(k + rank(d))` с `k=60` (стандартное значение из оригинальной статьи). RRF работает только с рангами, не требует нормализации скоров из разных систем. После RRF результаты агрегируются из child-чанков в parent-секции (`children_to_parents`) — берётся максимальный score среди children каждого parent.

### Векторная БД: ChromaDB

Персистентное хранилище (`PersistentClient`) с HNSW-индексом (cosine space). Выбор обусловлен: встроенная поддержка Python без сервера, метаданные хранятся вместе с эмбеддингами, батчевая вставка (по 500 документов). Архитектура **parent-child**: индексируются только child-чанки (300 символов, overlap 50), а при retrieval возвращаются полные parent-секции — это даёт точность мелких чанков при полноте больших контекстов.

### Чанкинг: `RecursiveCharacterTextSplitter`

Из `langchain-text-splitters`. Параметры: `chunk_size=300`, `chunk_overlap=50`, разделители `['. ', '; ', ', ', ' ', '']`. Рекурсивная стратегия сначала пытается разделить по предложениям (`. `), затем по всё более мелким единицам. Минимальная длина чанка — 40 символов (`MIN_CHUNK_LENGTH`). Размер 300 символов оптимален для `all-MiniLM-L6-v2` (max sequence 256 токенов ≈ 300–400 символов).

### LLM: `Qwen2-1.5B-Instruct` (по умолчанию)

По умолчанию ставим Qwen2-1.5B-Instruct: он достаточно умён, чтобы следовать структурированным инструкциям (XML-токены `<CONTEXT>`, `<SOURCE>`, `<QUESTION>`, `<INSTRUCTION>`) и цитировать источники в формате `[PMID:X]`. Поддерживает chat template (`apply_chat_template`), что упрощает формирование промпта. В каталоге также доступны:

| Модель | Размер | CPU RAM | Качество | Примечания |
|--------|--------|---------|----------|------------|
| TinyLlama 1.1B | 1.1B | ~5 GB | 2/5 | Самая лёгкая, низкое цитирование |
| **Qwen2 1.5B** | 1.5B | ~7 GB | 3/5 | **По умолчанию**, лучший для CPU |
| Phi-2 2.7B | 2.7B | ~11 GB | 3/5 | Нет chat template (plain prompt) |
| Gemma 2 2B | 2.6B | ~11 GB | 3/5 | Gated, нужен HF token |
| Mistral 7B | 7.2B | ~30 GB | 4/5 | Лучший баланс (GPU 8+ GB) |
| Qwen2 7B | 7.6B | ~32 GB | 4/5 | Отличное следование инструкциям |
| Llama 3.1 8B | 8.0B | ~34 GB | 5/5 | Лучшая faithfulness (gated) |

Поддерживается квантизация: FP16, 4-bit NF4 (bitsandbytes), 8-bit — для запуска крупных моделей на GPU с ограниченной VRAM.

### Почему не более крупная модель по умолчанию

Модели 7B+ дали бы лучшее качество генерации и цитирования, но требуют мощную видеокарту (8+ GB VRAM) или 30+ GB оперативки.


### Подсчёт токенов: токенизатор модели с tiktoken fallback

Для контроля бюджета токенов при сборке промпта используется `count_tokens()`: сначала пытается подсчитать через токенизатор загруженной LLM (`tokenizer.encode`) — это даёт точный результат для конкретной модели. Если модель ещё не загружена (например, на этапе сборки промпта до инициализации LLM), используется `tiktoken` с кодировкой `cl100k_base` как fallback.

### Structured Prompt: XML-токены

Для чёткого разделения контекста, вопроса и инструкций используются XML-подобные токены-разделители (`<CONTEXT>`, `<SOURCE pmid="..." section="..." year="...">`, `<QUESTION>`, `<INSTRUCTION>`). Модели (особенно Qwen, Llama, Mistral) обучены на данных с XML/HTML-разметкой и хорошо распознают структурные теги.

### Метрики качества: proxy-оценки без human evaluation

Пять автоматических метрик для оценки качества ответов:

| Метрика | Формула | Что измеряет |
|---------|---------|-------------|
| Faithfulness | cited_sentences / total_sentences | Доля предложений с цитатой `[PMID:X]` |
| Source Coverage | cited_pmids ∩ source_pmids / source_pmids | Какую долю найденных источников модель процитировала |
| Citation Accuracy | cited_pmids ∩ source_pmids / cited_pmids | Все ли цитаты ссылаются на реальные источники |
| Context Similarity | mean(chunk_similarity) | Средний retrieval score чанков |
| Answer Relevance | cos_sim(embed(query), embed(answer)) | Семантическая близость вопроса и ответа |
